<h1>Testing out Jupyter and yahoo finance API with pandas</h1>
    
Will be experimenting with this for API testing and playing with new concepts as well as taking notes. 

<b>Sources to look at in the future</b>
Using yahoo finance in most work below.  

Just came across the dividendhistory.org weekly reports which include P/E, P/B, payout ratio.  Need to writw up a quick scraper/collector to pull these down each week on Fridays. <br>
link: https://dividendhistory.org/report/ <br>
link US report: https://dividendhistory.org/reports/USA/2020-01-31-report.htm <br>
link CAN report: https://dividendhistory.org/reports/CAN/2020-01-31-report.htm <br>


To run a symbol through all code below, simply change variable <b>ticker</b> below and run all code. 


In [2]:
import pandas as pd
import pandas_datareader.data as pdr
import yfinance as yf

yf.pdr_override()
# global test parameters for workflow
ticker = 'ATD-B.TO'
start_date = '2010-01-01'
end_date = '2020-01-20'

yahoo_price_check = pdr.DataReader(ticker,  start=start_date, end=end_date, data_source='yahoo')
yahoo_price_check.reset_index(inplace=True,drop=False)
print(yahoo_price_check)


[*********************100%***********************]  1 of 1 completed
           Date       Open       High        Low      Close  Adj Close  \
0    2010-01-04   3.470000   3.480000   3.376670   3.386670   2.661294   
1    2010-01-05   3.401670   3.415000   3.333330   3.370000   2.648195   
2    2010-01-06   3.368330   3.381670   3.343330   3.358330   2.639024   
3    2010-01-07   3.340000   3.441670   3.335000   3.415000   2.683557   
4    2010-01-08   3.405000   3.415000   3.356670   3.370000   2.648195   
...         ...        ...        ...        ...        ...        ...   
2516 2020-01-14  42.959999  43.860001  42.799999  43.790001  43.790001   
2517 2020-01-15  43.759998  44.049999  43.119999  43.959999  43.959999   
2518 2020-01-16  44.049999  44.299999  43.790001  44.250000  44.250000   
2519 2020-01-17  44.290001  44.590000  43.980000  44.430000  44.430000   
2520 2020-01-20  44.430000  44.750000  44.299999  44.669998  44.669998   

       Volume  
0      800418  
1      765

    Alright so that is working ok.  Now lets get some dividend.  May need to try out adj close to account for splits
    
    so the date is used as index but cannot be referenced for joining later.  I reset the index to generic integer based index.  Doing thing on the price dataframe gives a regular Date column but for some reason, the same conversion leaves date labelled 'index' for yahoo-actions.  Quick relabel after reindexing should make this joinable. 

In [3]:
yf.pdr_override()

yahoo_div_check = pdr.DataReader(ticker, start=start_date, end=end_date, data_source='yahoo-actions')
yahoo_div_check.reset_index(inplace=True,drop=False)
yahoo_div_check = yahoo_div_check.rename(columns={'index':'Date'})
print(yahoo_div_check)

[*********************100%***********************]  1 of 1 completed
           Date       Open       High        Low      Close  Adj Close  \
0    2010-01-04   3.470000   3.480000   3.376670   3.386670   2.661294   
1    2010-01-05   3.401670   3.415000   3.333330   3.370000   2.648195   
2    2010-01-06   3.368330   3.381670   3.343330   3.358330   2.639024   
3    2010-01-07   3.340000   3.441670   3.335000   3.415000   2.683557   
4    2010-01-08   3.405000   3.415000   3.356670   3.370000   2.648195   
...         ...        ...        ...        ...        ...        ...   
2516 2020-01-14  42.959999  43.860001  42.799999  43.790001  43.790001   
2517 2020-01-15  43.759998  44.049999  43.119999  43.959999  43.959999   
2518 2020-01-16  44.049999  44.299999  43.790001  44.250000  44.250000   
2519 2020-01-17  44.290001  44.590000  43.980000  44.430000  44.430000   
2520 2020-01-20  44.430000  44.750000  44.299999  44.669998  44.669998   

       Volume  
0      800418  
1      765

OK so the yahoo-actions override is not working ... no worries.  yfinance refactor may be the better option.  
Test sections below.

In [4]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data
hist = msft.history(period="max")

# show actions (dividends, splits)
print(msft.actions)

# show dividends
msft.dividends

# show splits
msft.splits

# show financials
msft.financials
msft.quarterly_financials

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

# show balance heet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
#msft.isin

# show options expirations
#msft.options

# get option chain for specific expiration
#opt = msft.option_chain('YYYY-MM-DD')
# data available via: opt.calls, opt.puts

            Dividends  Stock Splits
Date                               
1987-09-21       0.00           2.0
1990-04-16       0.00           2.0
1991-06-27       0.00           1.5
1992-06-15       0.00           1.5
1994-05-23       0.00           2.0
...               ...           ...
2019-02-20       0.46           0.0
2019-05-15       0.46           0.0
2019-08-14       0.46           0.0
2019-11-20       0.51           0.0
2020-02-19       0.51           0.0

[74 rows x 2 columns]


,0,1
Earnings Date,2020-04-22 00:00:00,2020-04-27 00:00:00
Earnings Average,1.33,1.33
Earnings Low,1.3,1.3
Earnings High,1.43,1.43
Revenue Average,34606800000,34606800000
Revenue Low,34325200000,34325200000
Revenue High,35111000000,35111000000


Alright hitting a problem now with pandas datareader for yahoo.  Quick bit of research shows that using a library called yfinance (https://pypi.org/project/yfinance/) maybe the better option for accessing this data.  can install yfinance from pip and there is a function call that allows for pdr override with a single function call.  Maybe a painless fix.  

Also need to start exploring other API options as yahoo is deprecated and may become unreliable. 

<b> Playing with Graphing Data from Yahoo Finance </b>

In [5]:
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters



register_matplotlib_converters()

plt.scatter(yahoo_div_check['Date'], yahoo_div_check['value'], color = 'red')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Dividend Amount')
plt.show()

KeyError: 'value'

In [ ]:
#yahoo_div_check = pdr.DataReader(ticker, start='2004-01-01', end='2019-12-30', data_source='yahoo-actions')
print(yahoo_div_check)

Show all non-DIVIDEND actions. 

In [ ]:
yahoo_div_check = pdr.DataReader(ticker, start=start_date, end=end_date, data_source='yahoo-actions')
yahoo_div_check = yahoo_div_check[yahoo_div_check.action != "DIVIDEND"]
yahoo_div_check.reset_index(inplace=True,drop=False)
yahoo_div_check = yahoo_div_check.rename(columns={'index':'Date'})
print(yahoo_div_check)

So the outlier was a stock split not a dividend.  Should remove those.  Also whats happening pre-2002?

In [ ]:
yahoo_action_check = pdr.DataReader(ticker, start=start_date, end=end_date, data_source='yahoo-actions')
yahoo_div_check = yahoo_action_check[yahoo_action_check.action == "DIVIDEND"]
yahoo_split_check = yahoo_action_check[yahoo_action_check.action == "SPLIT"]
yahoo_other_check = yahoo_action_check[yahoo_action_check.action != "DIVIDEND"]

yahoo_div_check.reset_index(inplace=True,drop=False)
yahoo_div_check = yahoo_div_check.rename(columns={'index':'Date'})

yahoo_split_check.reset_index(inplace=True,drop=False)
yahoo_split_check = yahoo_split_check.rename(columns={'index':'Date'})

yahoo_other_check.reset_index(inplace=True,drop=False)
yahoo_other_check = yahoo_other_check.rename(columns={'index':'Date'})

print(yahoo_div_check)
print(yahoo_split_check)
print(yahoo_other_check)

<b>so got rid of splits ... only dividends ... now graph to check<b>

In [ ]:
plt.scatter(yahoo_div_check['Date'], yahoo_div_check['value'], color = 'red')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Dividend Amount')
plt.show()

This graph makes it easy to see whether a stock has a regular dividend payout and periodic increases.  

Drops in dividend payout (see ATD-B.TO) could be caused by stock splits.   Need to check for splits and adjust.

Now lets create a data frame with all the dividend yield values based on div and closing price by day.

In [ ]:
div_yield_frame = yahoo_price_check.copy()
#div_yield_frame = yahoo_price_check.loc[:,'Close']


div_yield_frame = div_yield_frame.drop('High', axis=1)
div_yield_frame = div_yield_frame.drop('Low', axis=1)
div_yield_frame = div_yield_frame.drop('Open', axis=1)
div_yield_frame = div_yield_frame.drop('Volume', axis=1)
div_yield_frame = div_yield_frame.drop('Adj Close', axis=1)
#div_yield_frame = div_yield_frame.drop('index', axis=1)



print(div_yield_frame)

So the merge does a join on date but only keeps results that match.  Need to populate all price closes rows with a dividend yield.  Try a different function.  Something that gives me an outer join (or right join)

currently fixed: auto filled non-div days with NaN.  

Next need to populate all the NaN spots with previous dividend value by date and calculate yield based on price each day. 

Alright now add a column for the yield for each day based on calculation . Dividend/price * 100

In [ ]:
#print(div_yield_frame)
#print(yahoo_div_check)
new_div_yield = pd.merge(div_yield_frame, yahoo_div_check, on="Date", how="left")
new_div_yield = new_div_yield.drop('action', axis=1)

# front fill and calculate yield for each day
# not sure if I need the backfill call to complete the dataframe or not
new_div_yield.fillna(method='ffill', inplace=True)
new_div_yield.fillna(method='bfill', inplace=True)
print(new_div_yield)
new_div_yield['div_yield'] = (4*new_div_yield['value'])/new_div_yield['Close'] *100
print(new_div_yield)

# plot dividend yield over time
plt.plot(new_div_yield['Date'], new_div_yield['div_yield'], color = 'red')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Dividend Yield')
plt.show()

# plot price over time 
plt.plot(new_div_yield['Date'], new_div_yield['Close'], color = 'green')
plt.title('Test Yahoo Dividends over time')
plt.xlabel('Date')
plt.ylabel('Price')
plt.show()

 ok great ... but the assumption here is that dividends are paid quarterly.  Need to calculate dividend frequency.  Maybe there is a way to get it.  if not, calculate it.  
 
<b>To do:</b> find a way to automatically calculate dividend payout frequency.  

   <i>Initial idea:</i>  for each dividend payment calculate delta between last and current div date and add to dividend yield DF.  This becomes a multiplier used to calculate div yield.  This should account for varying div payout frequencies across stocks <b>(test cases: T.TO, SJR-B.TO, XQQ)</b> as well as changes in payout frequency for individual stocks <b>(test case: Corus CJR-B)</b> 

hmm... so the closing price is adjusted for splits but the dividends are not.  Will need to account for dividend values that change as a result of splits.  To see this, change ticker to <b>(test case: "ATD-B.TO")</b>. 

<b>To do:</b> find a way to account for splits. 

<i>Initial idea:</i> So i think i can normalize by retroactively multiplying the dividend value by the split (eg  new_div = old_div * SPLIT) for each SPLIT record.  This would work for ATB-B.  will try later.  

<b>FALSE - need to adjust manually</b> Alternatively, sourcing dividend payouts from dividendhistory.org, the amounts are already adjusted but i have noticed some inconsistencies.  Going to try and account for splits.  

In [ ]:
for index, row in yahoo_split_check.iterrows():
    if not yahoo_split_check.empty:
        #print(row['Date'])
        test_div = new_div_yield.loc[new_div_yield['Date'] == row['Date']]
        print(test_div)
        print(row['value'])
        for index_2, row_2 in new_div_yield.iterrows():
            if row_2['Date']<=row['Date']:
                new_div_yield['value'] = new_div_yield['value']*row['value']
                #print("Old Div: {0}    New Div: {1}   Date: {2}  ".format(row_2['value'], row_2['Date']))
            #else:
                #print("no correction")
                
    else:
        print("No splits detected")
        


Once data is all adjusted and cleaned up - peak/trough finding/generation